In [5]:
import sys 
sys.path.append('..')

import pymongo
from bson.objectid import ObjectId

import utils

In [6]:
client = pymongo.MongoClient('10.157.90.169', 27017)
db = client['quarantine20']
db.collection_names()

/home/matt/config/py3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  This is separate from the ipykernel package so we can avoid doing imports until


['ba_paper',
 'percent_survived',
 'quarantine_by_props',
 'plc_qtines',
 'epidemics',
 'vanilla_runs',
 'graphs']

In [ ]:
len(list(db.percent_survived.aggregate([{'$group': {'_id': {'graph':'$graph_id', 'run':'$run_id', 'stop_prop': '$stop_prop'},'count': {'$sum': 1}}}])))

In [ ]:
run_id = db['percent_survived'].find_one()['run_id']

In [ ]:
def collect_perc_survived_plot(db, run_id, stop_prop):
    # Collects and aggregates data from the percent_survived plot 
    # output is a list of tuples [(start_degree, percent_survived), ...]
    docs = list(db['percent_survived'].find({'run_id': ObjectId(run_id), 'stop_prop': stop_prop}))
    survs = utils.mergesum([_['survived'] for _ in docs])
    outlist = [] 
    for k, v in sorted(survs.items(), key=lambda p: int(p[0])):
        outlist.append((int(k), v['final'] / v['original']))
    return outlist

In [ ]:
prop_list = [_ / 100 for _ in range(61)]

In [ ]:
data = {prop: collect_perc_survived_plot(db, run_id, prop) for prop in prop_list}

In [ ]:
%matplotlib notebook
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
x = range(10)
line, = ax.plot(x, np.sin(x))
ax.plot(range(1000), [0] * 1000, linestyle='--')
ax.set_xlim(0, 750)
ax.set_ylim(-0.1, 1.0)
ax.set_xlabel("Start Degree")
ax.set_ylabel("Percent surviving")


def update(w = 0.20):
    ax.set_title("Running until %s of pop. IR" % w)
    spo = data[w]
    line.set_xdata([_[0] for _ in spo])
    line.set_ydata([_[1] for _ in spo])
    fig.canvas.draw_idle()

interact(update, w=FloatSlider(value=0.2, min=0.00, max=0.6, step=0.01))

In [ ]:
#### SLIGHTLY LARGER HEATMAP 
list(db.quarantine_by_props.find(projection=['quarantine_props']))

In [ ]:
qprop_doc = db.quarantine_by_props.find_one()
graph_id = qprop_doc['graph_id']
epidemic_id = qprop_doc['epidemic_id']

def get_avg_IR(collection, graph_id, epidemic_id, filter_by='quarantine_props'):
    output_dict = {}
    for doc in collection.find({'graph_id': graph_id, 'epidemic_id': epidemic_id},
                               projection=['final_R', 'max_I', 'quarantine_props']):
        filter_key = doc[filter_by]
        if filter_key not in output_dict:
            output_dict[filter_key] = {'final_R': [], 'max_I': []}
        output_dict[filter_key]['final_R'].append(doc['final_R'])
        output_dict[filter_key]['max_I'].append(doc['max_I'])
        
    keydict = lambda d: {k: sum(v) / len(v) for k, v in d.items()}
    
    return {k: keydict(v) for k, v in output_dict.items()}
        
    


In [ ]:
avg_IRs = get_avg_IR(db.quarantine_by_props, graph_id, epidemic_id)
Xs, Is, Rs = [], [], [] 
for k in sorted(avg_IRs.keys()):
    Xs.append(k)
    Is.append(avg_IRs[k]['max_I'])
    Rs.append(avg_IRs[k]['final_R'])
    
    

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
ax.plot(Xs, Is, c='b')
ax.plot(Xs, Rs, c='g')